In [1]:
import json
import numpy as np
from pprint import pprint
idlists=[]
data = json.load(open('layer1.json'))
dets = json.load(open('det_ingrs.json','r'))
idx2ind = {}

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf


import numpy as np
for i,entry in enumerate(dets):
    idx2ind[entry['id']] = i
partition = str('train')

In [3]:
corpus=[]
allingrs=[]
idlists=[]
num=0
for i,entry in enumerate(data):
    if entry['partition'] == partition:
        ingrs=''
        idlists.append(entry['id'])
        det_ingrs = dets[idx2ind[entry['id']]]['ingredients']
        valid = dets[idx2ind[entry['id']]]['valid']    
        ingreds= entry['ingredients']
        allingreds=''
        for ingred in ingreds:
            ingred =  ingred['text']
            allingreds+=ingred +','

        for j,det_ingr in enumerate(det_ingrs):
        # if detected ingredient matches ingredient text,
        # means it did not work. We skip
            if not valid[j]:
                continue
        # underscore ingredient
            
            det_ingr_undrs = det_ingr['text'].replace(' ','_')
        
            ingrs+=det_ingr_undrs+' '
            allingrs.append(det_ingr_undrs)
            #allingreds = allingreds.replace(det_ingr['text'],det_ingr_undrs)
        corpus.append(ingrs)
        num+=1
        if (num==30000):
            break

In [6]:
#to vectorize the labels


vocabulary_size = 15000


def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    index = dictionary.get(word, 0)
    if index == 0:  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary




databag, count, dictionary, reverse_dictionary = build_dataset(allingrs,
                                                            vocabulary_size)

In [11]:

train_labels=np.zeros((len(corpus),len(count)),dtype=int)
startindex=0
fileDir = os.path.dirname(os.path.realpath('__file__'))
print (fileDir)
for x,idx in zip(corpus,idlists):
    filename = os.path.join(fileDir, 'image_labels_dir/'+idx+'.jpg.txt')
    f1 = open(filename,'w')   
    newcorpus=len(np.array(x.split(' ')))
    for i in range(startindex,startindex+newcorpus-1):
        f1.write(reverse_dictionary[databag[i]]+'\n')  
    startindex=newcorpus+1
    f1.close()

/home/shwetashanbhag/Project/Untitled Folder


In [17]:

f2= open('labels.txt','w')
for i in range(1,len(count)):
    f2.write(count[i][0]+'\n')
f2.close()